In [1]:
from transformers import pipeline, set_seed
import torch
import tensorflow as tf
generator = pipeline('text-generation', model='gpt2')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


Some weights of GPT2Model were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, I didn't want to work with another language like Rust. I'm making more time for Rust. I just"},
 {'generated_text': 'Hello, I\'m a language model, and I think this is a great time to look at how languages and their dependencies do it," he said.'},
 {'generated_text': "Hello, I'm a language model, and writing one is like writing a dictionary. I can put any number of objects together, and it keeps the"},
 {'generated_text': "Hello, I'm a language model, I'm a grammar model\n\nLinguistics [ edit ]\n\nThe term linguistics refers to the"},
 {'generated_text': "Hello, I'm a language model, not a system programming language. The language we were writing in Java was very basic and it was pretty much a"}]